# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- template script for analyzing modes spectra for sumatra
- downloads BHZ and LHZ data needed for hw_sumatraB

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from obspy import read
from obspy.core import UTCDateTime

from pysep import Pysep

from lib_seis import get_dist_az
from lib_seis import plot_event_station
from lib_seis import station_info_list
from lib_seis import wf_fft

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

origin time = 2004-12-26 00:58:53 \
latitude    = 3.09 N \
longitude   = 94.26 E \
depth       = 30 km \
magnitude   = 9.1

In [ ]:
# download raw LHZ data

download  = dict( client                       = 'IRIS',
                  origin_time                  = UTCDateTime('2004,12,26,00,58,53'),
                  event_latitude               = 3.09,
                  event_longitude              = 94.26,
                  event_depth_km               = 30,
                  event_magnitude              = 9.1,
                  networks                     = 'G,II,IU',
                  stations                     = '*',
                  locations                    = '*',
                  channels                     = 'LHZ',
                  remove_response              = False,
                  remove_clipped               = False,
                  remove_insufficient_length   = False,
                  detrend                      = False,
                  demean                       = False,
                  pre_filt                     = None,
                  fill_data_gaps               = 0,
                  gap_fraction                 = 1,
                  resample_freq                = 1,
                  seconds_before_ref           =  0.5 * 24 * 60 * 60,
                  seconds_after_ref            = 10.0 * 24 * 60 * 60,
                  taup_model                   = None,
                  log_level                    = 'CRITICAL',
                  write_files                  = 'sac',
                  plot_files                   = 'map',
                  output_dir                   = 'datawf',
                  overwrite_event_tag          = 'sumatra_LHZ_raw')

try:
    ps = Pysep(**download)
    ps.run()
except :
    print('data directory already exists')

In [ ]:
# read downloaded raw LHZ data

sac_file = f'./datawf/sumatra_LHZ_raw/SAC/*.sac'
st       = read(sac_file, header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# waveforms to reject because of poor data quality

waveforms_to_reject  = ['G.PEL..LHZ',
                        'II.DGAR.00.LHZ',
                        'II.HOPE.00.LHZ',
                        'II.PALK.00.LHZ',
                        'II.SHEL.00.LHZ',
                        'IU.ADK.00.LHZ',
                        'IU.DAV.00.LHZ',
                        'IU.FUNA.00.LHZ',                  
                        'IU.FUNA.10.LHZ',
                        'IU.GRFO..LHZ',
                        'IU.LCO..LHZ',
                        'IU.OTAV.00.LHZ',
                        'IU.OTAV.10.LHZ',
                        'IU.PMG.00.LHZ',
                        'IU.PMG.10.LHZ',
                        'IU.POHA.00.LHZ',
                        'IU.PTCN.00.LHZ',
                        'IU.MBWA.00.LHZ',
                        'IU.RAO.00.LHZ',
                        'IU.RSSD.00.LHZ',
                        'IU.SAML.00.LHZ',
                        'IU.SAML.10.LHZ',
                        'IU.SDV.00.LHZ',
                        'IU.SDV.10.LHZ',
                        'IU.TRIS.00.LHZ',
                        'IU.TRIS.10.LHZ',
                        'IU.WAKE.00.LHZ',
                        'IU.XMAS.00.LHZ']

In [ ]:
# plot rejected waveforms

for i, waveform_id in enumerate(waveforms_to_reject):
    
    tr = st.select(id=waveform_id)
    plt.figure()
    plt.plot(np.where(tr[0].data==0.00000000, np.nan, tr[0].data))                                         # replacing filled gaps (0's) with gaps (Nan's)
    plt.xlim(0,907200)
    plt.title(f'{tr[0].id} − starting {tr[0].stats["starttime"]}')

In [ ]:
# special trimming of waveforms which need it
# ID | starttime | endtime

trim_details = [['IU.QSPA.20.LHZ', 0       , 4.4800e5],
                ['II.ABKT.00.LHZ', 0       , 6.7410e5],
                ['G.ATD.20.LHZ'  , 0       , 2.8400e5],
                ['IU.BILL.20.LHZ', 0       , 4.3050e5],
                ['II.BORG.20.LHZ', 0       , 3.1700e5],
                ['IU.CHTO.20.LHZ', 0       , 3.6270e5],
                ['II.EFI.20.LHZ' , 0       , 2.7500e5],
                ['IU.GUMO.20.LHZ', 0       , 1.7980e5],
                ['IU.HRV.20.LHZ' , 0       , 4.5800e5],
                ['IU.MIDW.20.LHZ', 0.7500e4, 9.0720e5],
                ['II.PFO.20.LHZ' , 0       , 3.3205e5],
                ['G.SCZ.20.LHZ'  , 0       , 1.9300e5],
                ['G.TAM.20.LHZ'  , 0       , 5.5800e5],
                ['IU.TIXI.20.LHZ', 0       , 6.6150e5],
                ['G.WUS.20.LHZ'  , 0       , 5.8200e5]]

In [ ]:
# computing and plotting the discrete Fourier transforms of the non-removed waveforms

# resample_freq is provided as input into pysep (see above)
Nyquist_frequency = download['resample_freq']/2

for i in range(len(st)):
    
    if st[i].id in waveforms_to_reject:
        continue
    
    else:
        tr = st[i].copy()
        
        # cut out specific sections of specific seismograms
        if tr.id in trim_details[:][0]:
            start_time = tr.stats.starttime + trim_details[trim_details[:][0].index(tr.id)][1] 
            end_time   = tr.stats.starttime + trim_details[trim_details[:][0].index(tr.id)][2]
            tr.trim(start_time,end_time)
          
        tr.detrend('constant')
        tr.detrend('linear')
        tr.taper(max_percentage=0.2, type="cosine")
        
        fft_amp, fft_phase, fft_freq = wf_fft(tr.data, Nyquist_frequency)
        
        plt.figure()
        plt.plot(fft_freq*1E3,fft_amp)
        plt.xlim(0.2,1)
        #plt.ylim(0,40)
        xmask = np.logical_and(fft_freq*1e3 > .2, fft_freq*1e3 < 1)
        plt.ylim(0,np.max(fft_amp[xmask])*1.1)
        plt.title(f'{tr.id}')
        plt.xlabel(f'Frequency (mHz)')
        plt.ylabel(f'Amplitude (counts s)')
        plt.show()        

In [ ]:
# pick at least 20 waveforms having a useful and clean amplitude spectrum
# pick only one location for a given station

# list of stations with sensors at multiple locations
# network = G  - KIP
# network = II - ALE, ASCN, BORG, DGAR, KDAK, PFO, RPN, WRAB
# network = IU - AFI, ANMO, CASY, COLA, FURI, GUMO, HNR, INCN, KIP, QSPA
#                RAO, RAR, RCBR, SAML, SDV, SNZO, TATO, TEIG, TRIS, TRQA, WAKE

# network | station | location | channel

picked_waveforms = [['G' ,'CAN' ,''  ,'LHZ'],
                    ['G' ,'CAN' ,''  ,'LHZ'],
                    ['G' ,'CAN' ,''  ,'LHZ']]

In [ ]:
# storing station information for selected stations
station_lats, station_lons, station_tags = station_info_list(st, picked_waveforms)

In [ ]:
print('\nSource receiver map')   

event_lon = download['event_longitude']
event_lat = download['event_latitude']

plot_event_station(event_lat,event_lon,slat=station_lats,slon=station_lons,stas=station_tags)

In [ ]:
print('Table of station azimuths and distances\n')

distance_deg, azimuth_deg, distance_km = get_dist_az(event_lon,event_lat,station_lons,station_lats,station_tags)

In [ ]:
# download LHZ data for selected waveforms - response removed to acceleration

for i, waveform_id in enumerate(picked_waveforms):

    download  = dict( client                       = 'IRIS',
                      origin_time                  = UTCDateTime("2004,12,26,00,58,53"),
                      event_latitude               = 3.09,
                      event_longitude              = 94.26,
                      event_depth_km               = 30,
                      event_magnitude              = 9.1,
                      networks                     = waveform_id[0],
                      stations                     = waveform_id[1],
                      locations                    = waveform_id[2],
                      channels                     = waveform_id[3],
                      remove_response              = True,
                      output_unit                  = 'ACC',
                      remove_clipped               = True,
                      remove_insufficient_length   = True,
                      detrend                      = True,
                      demean                       = True,
                      pre_filt                     = None,
                      fill_data_gaps               = 0,
                      gap_fraction                 = 0.03,
                      resample_freq                = 1,
                      seconds_before_ref           =  0.5 * 24 * 60 * 60,
                      seconds_after_ref            = 10.0 * 24 * 60 * 60,
                      taup_model                   = None,
                      log_level                    = 'DEBUG',
                      write_files                  = 'sac',
                      plot_files                   = None,
                      output_dir                   = 'datawf',
                      overwrite                    = True,
                      overwrite_event_tag          = 'sumatra_LHZ_acc')
    
    try:
        ps = Pysep(**download)
        ps.run()
    except:
        print("Check the correctness of picked waveforms' list definition. If that is not a solution, check PySEP's debug messages.")    

In [ ]:
# download BHZ data - response removed to displacement

download  = dict( client                       = 'IRIS',
                  origin_time                  = UTCDateTime("2004,12,26,00,58,53"),
                  event_latitude               = 3.09,
                  event_longitude              = 94.26,
                  event_depth_km               = 30,
                  event_magnitude              = 9.1,
                  networks                     = 'G,II,IU',
                  stations                     = '*',
                  locations                    = '*',
                  channels                     = 'BHZ',
                  remove_response              = True,
                  output_unit                  = 'DISP',
                  remove_clipped               = True,
                  remove_insufficient_length   = True,
                  detrend                      = True,
                  demean                       = True,
                  pre_filt                     = None,
                  fill_data_gaps               = 0,
                  gap_fraction                 = 0.03,
                  resample_freq                = 20,
                  seconds_before_ref           = 1 * 60 * 60,
                  seconds_after_ref            = 4 * 60 * 60,
                  taup_model                   = None,
                  log_level                    = 'CRITICAL',
                  write_files                  = 'sac',
                  plot_files                   = 'map',
                  output_dir                   = 'datawf', 
                  overwrite_event_tag          = 'sumatra_BHZ_disp')

ps = Pysep(**download)
ps.run()